In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
import numpy as np

In [2]:
text = '해보지 않으면 해낼 수 없다'
result = text_to_word_sequence(text)
print(result)

['해보지', '않으면', '해낼', '수', '없다']


In [3]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

token = Tokenizer()
token.fit_on_texts(docs)

print(token.word_counts) #해당 분리된 부분이 몇개의 단어로 이루어졌는가

OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [4]:
print(token.document_count) #문장의 개수

3


In [5]:
print(token.word_docs) #해당단어가 몇개의 문장에 등장하는가

defaultdict(<class 'int'>, {'먼저': 1, '텍스트의': 2, '나누어': 1, '단어를': 1, '합니다': 1, '토큰화': 1, '각': 1, '단어로': 1, '인식됩니다': 1, '딥러닝에서': 2, '토큰화해야': 1, '토큰화한': 1, '사용할': 1, '수': 1, '있습니다': 1, '결과는': 1})


In [6]:
print(token.word_index)

{'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


In [7]:
text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [8]:
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [9]:
word_size = len(token.word_index) + 1
x = to_categorical(x, word_size)
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


In [10]:
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
classes = np.array([1,1,1,1,1,0,0,0,0,0])

# 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [11]:
x = token.texts_to_sequences(docs)
print(x)

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [12]:
padded_x = pad_sequences(x, 4)
print(padded_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [13]:
word_size = len(token.word_index) + 1
print('word_size: ', word_size)
# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력합니다.
model = models.Sequential()
model.add(layers.Embedding(word_size, 8, input_length=4))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

word_size:  21
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              168       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 632ms/step - loss: 0.6850 - accuracy: 0.8000
Epoch 2/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6828 - accuracy: 0.8000
Epoch 3/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6806 - accuracy: 0.8000
Epoch 4/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6785 - accuracy: 0.8000
Epoch 5/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6763 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6741 - accuracy: 0.9000
Epoch 7/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6720 - accuracy: 0.9000
Epoch 8/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6698 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6676 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6655 - accuracy: 0.9000
Epoch 11/20
1/1 [====================